In [1]:
import pandas as pd

# Configuração para acesso ao MinIO
s3_url = "http://54.161.237.227:9000/"
access_key = "minio_access_key"
secret_key = "minio_secret_key"

storage_options = {
    "key": access_key,
    "secret": secret_key,
    "client_kwargs": {
        "endpoint_url": s3_url
    }
}

file_path = 's3://bronze/data/categories.csv'

# Tentar ler o arquivo CSV com tratamento para aspas e delimitadores, e lidando com linhas ruins
try:
    df = pd.read_csv(
        file_path,
        sep=',',
        quotechar='"',  # Assegura que as vírgulas dentro de aspas sejam tratadas corretamente
        storage_options=storage_options,
        on_bad_lines='skip'  # Pula linhas que não têm o número correto de delimitadores
    )
    display(df)
except Exception as e:
    print("Erro ao ler o arquivo:", e)


,categoryID,categoryName,description,picture
0,1,Beverages,"Soft drinks, coffees, teas, beers, and ales",0x151C2F00020000000D000E0014002100FFFFFFFF4269...
1,2,Condiments,"Sweet and savory sauces, relishes, spreads, an...",0x151C2F00020000000D000E0014002100FFFFFFFF4269...
2,3,Confections,"Desserts, candies, and sweet breads",0x151C2F00020000000D000E0014002100FFFFFFFF4269...
3,4,Dairy Products,Cheeses,0x151C2F00020000000D000E0014002100FFFFFFFF4269...
4,5,Grains/Cereals,"Breads, crackers, pasta, and cereal",0x151C2F00020000000D000E0014002100FFFFFFFF4269...
5,6,Meat/Poultry,Prepared meats,0x151C2F00020000000D000E0014002100FFFFFFFF4269...
6,7,Produce,Dried fruit and bean curd,0x151C2F00020000000D000E0014002100FFFFFFFF4269...
7,8,Seafood,Seaweed and fish,0x151C2F00020000000D000E0014002100FFFFFFFF4269...


In [7]:
df_renamed = df.rename(columns={
    'categoryID': 'id_category', 
    'categoryName': 'ds_category_name',
    'description': 'ds_description',
    'picture': 'cd_picture' 
                   })

In [9]:
display(df_renamed)

,id_category,ds_category_name,ds_description,cd_picture
0,1,Beverages,"Soft drinks, coffees, teas, beers, and ales",0x151C2F00020000000D000E0014002100FFFFFFFF4269...
1,2,Condiments,"Sweet and savory sauces, relishes, spreads, an...",0x151C2F00020000000D000E0014002100FFFFFFFF4269...
2,3,Confections,"Desserts, candies, and sweet breads",0x151C2F00020000000D000E0014002100FFFFFFFF4269...
3,4,Dairy Products,Cheeses,0x151C2F00020000000D000E0014002100FFFFFFFF4269...
4,5,Grains/Cereals,"Breads, crackers, pasta, and cereal",0x151C2F00020000000D000E0014002100FFFFFFFF4269...
5,6,Meat/Poultry,Prepared meats,0x151C2F00020000000D000E0014002100FFFFFFFF4269...
6,7,Produce,Dried fruit and bean curd,0x151C2F00020000000D000E0014002100FFFFFFFF4269...
7,8,Seafood,Seaweed and fish,0x151C2F00020000000D000E0014002100FFFFFFFF4269...


In [14]:
df_formatted = df_renamed.astype({
    'id_category': 'int',
    'ds_category_name': 'string',
    'ds_description': 'string',
    'cd_picture': 'string',
})

display(df_formatted)

,id_category,ds_category_name,ds_description,cd_picture
0,1,Beverages,"Soft drinks, coffees, teas, beers, and ales",0x151C2F00020000000D000E0014002100FFFFFFFF4269...
1,2,Condiments,"Sweet and savory sauces, relishes, spreads, an...",0x151C2F00020000000D000E0014002100FFFFFFFF4269...
2,3,Confections,"Desserts, candies, and sweet breads",0x151C2F00020000000D000E0014002100FFFFFFFF4269...
3,4,Dairy Products,Cheeses,0x151C2F00020000000D000E0014002100FFFFFFFF4269...
4,5,Grains/Cereals,"Breads, crackers, pasta, and cereal",0x151C2F00020000000D000E0014002100FFFFFFFF4269...
5,6,Meat/Poultry,Prepared meats,0x151C2F00020000000D000E0014002100FFFFFFFF4269...
6,7,Produce,Dried fruit and bean curd,0x151C2F00020000000D000E0014002100FFFFFFFF4269...
7,8,Seafood,Seaweed and fish,0x151C2F00020000000D000E0014002100FFFFFFFF4269...


In [15]:
parquet_path = 's3://silver/categories.parquet'


try:
    df.to_parquet(
        parquet_path,
        engine='pyarrow',
        index=False,       
        storage_options=storage_options
    )
    print("Arquivo Parquet salvo com sucesso em:", parquet_path)
except Exception as e:
    print("Erro ao salvar o arquivo Parquet:", e)


Arquivo Parquet salvo com sucesso em: s3://silver/categories.parquet
